<a href="https://colab.research.google.com/github/psb-david-petty/google-colaboratory/blob/master/euler100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Euler 100

This is a look at a solution to [Project Euler #100](https://projecteuler.net/problem=100). The text reads:

> *If a box contains twenty-one coloured discs, composed of fifteen blue discs and six red discs, and two discs were taken at random, it can be seen that the probability of taking two blue discs, P(BB) = (15/21)×(14/20) = 1/2.*<br>
>
> *The next such arrangement, for which there is exactly 50% chance of taking two blue discs at random, is a box containing eighty-five blue discs and thirty-five red discs.*<br>
>
> *By finding the first arrangement to contain over 10 ** 12 = 1,000,000,000,000 discs in total, determine the number of blue discs that the box would contain.*

This amounts to finding values $blue$ and $red$ where $2\left[ blue(blue - 1) \right] = (blue + red)\times (blue + red - 1)$ where $(blue + red) > 10^{12}$.

## Na&#239;ve approach

The first na&#239;ve solution is a brute-force $O\left( n^{2} \right)$ approach, which works for the first thousand tests, but is completely inadequate for a trilion (even if, since $blue > red$ for this problem, a [triangular](https://en.wikipedia.org/wiki/Quadratic_growth) search will work).

In [15]:
#!/usr/env python3
#
# euler100.py
#

"""If a box contains twenty-one coloured discs, composed of fifteen blue discs and six red discs, and two discs were taken at random, it can be seen that the probability of taking two blue discs, P(BB) = (15/21)×(14/20) = 1/2.
The next such arrangement, for which there is exactly 50% chance of taking two blue discs at random, is a box containing eighty-five blue discs and thirty-five red discs.
By finding the first arrangement to contain over 10 ** 12 = 1,000,000,000,000 discs in total, determine the number of blue discs that the box would contain."""

# Quadratic brute-force solution.
top = 10 ** 3
for red in range(top):
    for blue in range(red, top):
        if 2 * blue * (blue - 1) == (blue + red) * (blue + red - 1):
            print(blue, red)

0 0
1 0
3 1
15 6
85 35
493 204


## Improve to *O(n)*

In order to improve the performance, notice that the ratio $\frac{\text{blue}}{\text{red}}$ approaches the [Silver Ratio](https://en.wikipedia.org/wiki/Silver_ratio): $\left( 1 + \sqrt{2} \right)$, so only look for $blue$ values around $\left[ red \times \left(1 + \sqrt{2}\right) \right]$.

Even with this $O\left( n^{2} \right)$ &rarr; $O\left( n \right)$ improvement, running $100,000$ tests took &#8776;3 seconds, $1,000,000$ tests took &#8776;30 seconds, so $1,000,000 \times 1,000,000$ tests will take &#8776;$30,000,000$s (which is &#8776;347 days) on an M1 Mac (&#8776;$40,000,000$s or &#8776;463 days, on this [Google Colaboratory](https://colab.research.google.com/) notebook).

In [16]:
#!/usr/env python3
#
# euler100.py
#
import math, time

"""If a box contains twenty-one coloured discs, composed of fifteen blue discs and six red discs, and two discs were taken at random, it can be seen that the probability of taking two blue discs, P(BB) = (15/21)×(14/20) = 1/2.
The next such arrangement, for which there is exactly 50% chance of taking two blue discs at random, is a box containing eighty-five blue discs and thirty-five red discs.
By finding the first arrangement to contain over 10 ** 12 = 1,000,000,000,000 discs in total, determine the number of blue discs that the box would contain."""

# The blue / red ratio approaches the silver ratio = 1 + math.sqrt(2)
silver = 1 + math.sqrt(2)
# Run this algorithm for 10 ** 2 to 10 ** 6.
for e in range(2, 7):
    top, start_time = 10 ** e, time.time()
    for red in range(2, top):
        # Increase / decrease the silver ratio by delta to calculate blue.
        delta = 10 ** - int(math.log(red, 10) - 1)
        start, end = red * (silver - delta), red * (silver + delta)
        # Make sure start < blue < end.
        if not (start < red * silver < end):
            print(f"{red * silver:.2f} not on [{int(start)},{int(end)}]")
        # Look at blue values around red * silver.
        for blue in range(int(start), int(end)):
            # if blue * blue - blue == red * red + 2 * red * blue - red:
            if 2 * blue * (blue - 1) == (blue + red) * (blue + red - 1):
                print(f"{blue} {red} ({blue} on [{int(start)},{int(end)}])")
    print(f"* time: {int(time.time() - start_time)}s for {top} tests")
# 31s for 10 ** 6 tests, 31 * 10 ** 6s (359 days) for 10 ** 12 tests.


15 6 (15 on [8,20])
85 35 (85 on [49,119])
* time: 0s for 100 tests
15 6 (15 on [8,20])
85 35 (85 on [49,119])
493 204 (493 on [472,512])
* time: 0s for 1000 tests
15 6 (15 on [8,20])
85 35 (85 on [49,119])
493 204 (493 on [472,512])
2871 1189 (2871 on [2858,2882])
16731 6930 (16731 on [16661,16799])
* time: 0s for 10000 tests
15 6 (15 on [8,20])
85 35 (85 on [49,119])
493 204 (493 on [472,512])
2871 1189 (2871 on [2858,2882])
16731 6930 (16731 on [16661,16799])
97513 40391 (97513 on [97472,97552])
* time: 4s for 100000 tests
15 6 (15 on [8,20])
85 35 (85 on [49,119])
493 204 (493 on [472,512])
2871 1189 (2871 on [2858,2882])
16731 6930 (16731 on [16661,16799])
97513 40391 (97513 on [97472,97552])
568345 235416 (568345 on [568320,568368])
* time: 42s for 1000000 tests


## A recursive and closed-form solution

OK, so waiting a year for the soultion is not an option&hellip;

Once some solutions are known (15, 85, 493, 2871, 16731, 97513, 568345), you can use the [OEIS](https://oeis.org/) to research others. 

https://oeis.org/search?q=15+85+493+2871+16731+97513+568345 shows one match: [A011900](https://oeis.org/A011900): *Members of Diophantine Pairs* &mdash; one element of solutions to [Pell's Equation](https://en.wikipedia.org/wiki/Pell's_equation) for $n = 2$. 

https://oeis.org/search?q=21+120+697+4060+23661+137904+803761 shows one match: [A046090](https://oeis.org/A046090): X+1 values of *Pythagorean triples (X,X+1,Z) ordered by increasing Z* &mdash; the total discs in the box.

[A011900](https://oeis.org/A011900) and [A046090](https://oeis.org/A046090) give both recursive definitions and closed-form formulas, so it is easy to solve for the first `a046090(n)` $> 1,000,000,000,000$ which is when $n = 16$.

It is interesting that both [A011900](https://oeis.org/A011900) (the blue values) and [A046090](https://oeis.org/A046090) (the totals for discs in the box) have the same recursive definitions, but with different base cases. 

### Solution

**So, if you have 756,872,327,473 blue discs and 313,506,783,024 red discs in a box, the probability of drawing two blue discs at random from the box is exactly 50%!**

### Epilog

If these trillion chips are standard $39\text{mm}$ poker chips where $11$ fit in a $39\text{mm}$ cube then, since $1070379110497 \approx 4600 \times 4600 \times 4600 \times 11$, the box containing these chips must be a $4600 \times 39\text{mm} \div 1000 = 179.4\text{m}$ cube which is $\approx 598$ feet on a side!

In [17]:
#!/usr/env python3
#
# euler100.py
#
import math, time

"""If a box contains twenty-one coloured discs, composed of fifteen blue discs and six red discs, and two discs were taken at random, it can be seen that the probability of taking two blue discs, P(BB) = (15/21)×(14/20) = 1/2.
The next such arrangement, for which there is exactly 50% chance of taking two blue discs at random, is a box containing eighty-five blue discs and thirty-five red discs.
By finding the first arrangement to contain over 10 ** 12 = 1,000,000,000,000 discs in total, determine the number of blue discs that the box would contain."""

# https://oeis.org/search?q=15+85+493+2871+16731+97513+568345
def a011900(n):
    """Return nth element of OEIS A011900."""
    if n == 0: return 1
    if n == 1: return 3
    return 6 * a011900(n - 1) - a011900(n - 2) - 2

# https://oeis.org/search?q=21+120+697+4060+23661+137904+803761
def a046090(n):
    """Return nth element of OEIS A046090."""
    if n == 0: return 1
    if n == 1: return 4
    return 6 * a046090(n - 1) - a046090(n - 2) - 2

solutions = 20
print([ a011900(i) for i in range(solutions) if a046090(i) > 10 ** 12 ][0])
print([ a011900(i) for i in range(solutions) ])
print([ a046090(i) - a011900(i) for i in range(solutions) ])

# a011900(n) as a closed-form expression.
a011900 = lambda n: int(round((((1 + math.sqrt(2)) ** (2 * n - 1) - (1 - math.sqrt(2)) ** (2 * n - 1)) / math.sqrt(8) + 1) / 2, 0))
print([ a011900(i) for i in range(solutions) ])
# a046090(n) as a closed-form expression function of a011900(n).
a046090 = lambda n: int((math.sqrt(1 + 8 * a011900(n) * (a011900(n) + 1))) / 2 - a011900(n))
print([ a046090(i) for i in range(solutions) ])
# These don't exactly match for n < 2 and the indexes are off by 1...


756872327473
[1, 3, 15, 85, 493, 2871, 16731, 97513, 568345, 3312555, 19306983, 112529341, 655869061, 3822685023, 22280241075, 129858761425, 756872327473, 4411375203411, 25711378892991, 149856898154533]
[0, 1, 6, 35, 204, 1189, 6930, 40391, 235416, 1372105, 7997214, 46611179, 271669860, 1583407981, 9228778026, 53789260175, 313506783024, 1827251437969, 10650001844790, 62072759630771]
[1, 1, 3, 15, 85, 493, 2871, 16731, 97513, 568345, 3312555, 19306983, 112529341, 655869061, 3822685023, 22280241075, 129858761425, 756872327473, 4411375203411, 25711378892991]
[1, 1, 1, 6, 35, 204, 1189, 6930, 40391, 235416, 1372105, 7997214, 46611179, 271669860, 1583407981, 9228778026, 53789260175, 313506783024, 1827251437969, 10650001844790]
